# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,2.08,86,99,2.80,RU,2024-08-27 19:42:13
1,1,port saint john's,-31.6229,29.5448,21.18,57,0,8.51,ZA,2024-08-27 19:42:14
2,2,punta arenas,-53.1500,-70.9167,8.06,61,75,9.77,CL,2024-08-27 19:37:34
3,3,wailua homesteads,22.0669,-159.3780,27.44,74,75,5.66,US,2024-08-27 19:42:16
4,4,blackmans bay,-43.0167,147.3167,8.07,89,99,0.89,AU,2024-08-27 19:42:18


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'EsriTerrain',
    coastline = True,
    size = 'Humidity',
    color = 'City',
    cmap='bmw',
    height = 800,
    width = 1400
)

# Display the map
city_data_map

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity)
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria are latitude between -30 and 30, and a humidity level between 30% and 40%
ideal_weather_df = city_data_df.loc[(city_data_df['Lat'] >= -30) & (city_data_df['Lat'] <= 30) & \
                        (city_data_df['Humidity'] <= 50) & (city_data_df['Humidity'] >= 30), :]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,al battaliyah,25.4333,49.6333,37.23,34,3,3.00,SA,2024-08-27 19:43:30
65,65,amancio,20.8257,-77.5799,33.37,48,95,3.65,CU,2024-08-27 19:43:31
74,74,tiznit,29.5833,-9.5000,23.16,45,0,1.53,MA,2024-08-27 19:43:41
84,84,kalemie,-5.9475,29.1947,24.64,48,21,1.00,CD,2024-08-27 19:43:53
108,108,mapimi,25.8167,-103.8500,30.56,31,0,3.64,MX,2024-08-27 19:44:22
148,148,menaka,15.9182,2.4022,34.59,39,68,6.39,ML,2024-08-27 19:45:10
160,160,choix,26.7092,-108.3219,35.45,41,99,2.06,MX,2024-08-27 19:45:25
171,171,san patricio,28.0170,-97.5169,33.97,45,100,3.42,US,2024-08-27 19:45:38
194,194,kununurra,-15.7667,128.7333,27.99,32,96,3.60,AU,2024-08-27 19:46:06
222,222,dolores hidalgo cuna de la independencia nacional,21.1561,-100.9308,26.58,35,0,3.66,MX,2024-08-27 19:42:47


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({'City': ideal_weather_df['City'], 
                         'Country': ideal_weather_df['Country'], 
                         'Lat': ideal_weather_df['Lat'], 
                         'Lng': ideal_weather_df['Lng'], 
                         'Humidity': ideal_weather_df['Humidity']})

# Add an empty column, 'Hotel Name,' to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
64,al battaliyah,SA,25.4333,49.6333,34,None
65,amancio,CU,20.8257,-77.5799,48,None
74,tiznit,MA,29.5833,-9.5000,45,None
84,kalemie,CD,-5.9475,29.1947,48,None
108,mapimi,MX,25.8167,-103.8500,31,None
148,menaka,ML,15.9182,2.4022,39,None
160,choix,MX,26.7092,-108.3219,41,None
171,san patricio,US,28.0170,-97.5169,45,None
194,kununurra,AU,-15.7667,128.7333,32,None
222,dolores hidalgo cuna de la independencia nacional,MX,21.1561,-100.9308,35,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    'categories':categories,
    'limit':limit,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print('Starting hotel search')

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lng},{lat},{radius}'
    params['bias'] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = 'https://api.geoapify.com/v2/places'

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as 'No hotel found'.
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'

    # Log the search results
    print(f'{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}')

# Display sample data
hotel_df

Starting hotel search
al battaliyah - nearest hotel: Grand Day Hotel
amancio - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
kalemie - nearest hotel: Hotel du Lac
mapimi - nearest hotel: No hotel found
menaka - nearest hotel: No hotel found
choix - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
dolores hidalgo cuna de la independencia nacional - nearest hotel: Hotel CasaMía
ciudad melchor muzquiz - nearest hotel: No hotel found
robore - nearest hotel: Lajas
minas de marcona - nearest hotel: No hotel found
az zawr - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
djambala - nearest hotel: No hotel found
pangoa - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
lodwar - nearest hotel: Lodwar Modern Lodge
san pedro pochutla - nearest hotel: Olimpia
quemado de guines - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
lagunas - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
64,al battaliyah,SA,25.4333,49.6333,34,Grand Day Hotel
65,amancio,CU,20.8257,-77.5799,48,No hotel found
74,tiznit,MA,29.5833,-9.5000,45,No hotel found
84,kalemie,CD,-5.9475,29.1947,48,Hotel du Lac
108,mapimi,MX,25.8167,-103.8500,31,No hotel found
148,menaka,ML,15.9182,2.4022,39,No hotel found
160,choix,MX,26.7092,-108.3219,41,No hotel found
171,san patricio,US,28.0170,-97.5169,45,No hotel found
194,kununurra,AU,-15.7667,128.7333,32,No hotel found
222,dolores hidalgo cuna de la independencia nacional,MX,21.1561,-100.9308,35,Hotel CasaMía


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display
normalized_humidity = (hotel_df['Humidity'] - hotel_df['Humidity'].min()) / (hotel_df['Humidity'].max() - hotel_df['Humidity'].min())
# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'CartoEco',
    coastline = True,
    size = 100,
    color = 'City',
    cmap = 'Cool',
    height = 550,
    width = 1200,
    hover_cols = ['Country', 'Hotel Name']
)

# Display the map
hotel_data_map


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Country,Hotel Name)
   .Coastline.I :Feature   [Longitude,Latitude]